In [1]:
import pandas as pd
import requests
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from time import sleep
from IPython.display import clear_output

def fix_zero_spikes(series):
    fixed = series.copy()
    for i in range(1, len(series) - 1):
        if series[i] == 0:
            if series[i-1] != 0:
                fixed[i] = series[i-1]
            elif series[i+1] != 0:
                fixed[i] = series[i+1]
    return fixed

# ThingSpeak channel parameters
CHANNEL_ID = '2505616'
READ_API_KEY = '0NK4IZ85O9HT42NZ'
RESULTS = 40

# Field names for labeling
FIELD_NAMES = {
    'field1': 'Temperature',
    'field2': 'Relative Humidity (rH)',
    'field3': 'PMS1',
    'field4': 'PMS2.5',
    'field5': 'PMS10'
}

def fetch_all_fields():
    url = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?results={RESULTS}&api_key={READ_API_KEY}"
    response = requests.get(url)
    data = response.json()
    feeds = data['feeds']
    df = pd.DataFrame(feeds)
    df['created_at'] = pd.to_datetime(df['created_at'])

    for i in range(1, 6):
        df[f'field{i}'] = pd.to_numeric(df[f'field{i}'], errors='coerce')
        df[f'field{i}'] = fix_zero_spikes(df[f'field{i}'])

    return df[['created_at'] + list(FIELD_NAMES.keys())].dropna()

def live_plot_combined(cycles=10, interval=15):
    for _ in range(cycles):
        df = fetch_all_fields()
        clear_output(wait=True)

        fig = make_subplots(
            rows=5, cols=1, shared_xaxes=True,
            subplot_titles=[FIELD_NAMES[f'field{i}'] for i in range(1, 6)]
        )

        for i in range(1, 6):
            field = f'field{i}'
            fig.add_trace(go.Scatter(
                x=df['created_at'],
                y=df[field],
                mode='lines',
                name=FIELD_NAMES[field]
            ), row=i, col=1)

        fig.update_layout(
            height=1400,
            title_text="Live ThingSpeak Data (Sensor Readings)",
            showlegend=False
        )
        fig.update_xaxes(title_text="Time", row=5, col=1)
        fig.update_yaxes(title_text="Sensor Value")

        fig.show()
        sleep(interval)

# Run the combined live plot
live_plot_combined(cycles=10, interval=15)
